# Semantic Link Data Preparation - Late Delivery Prediction

**Goal:** Prepare data from the "DLV Aging Columns & Measures" semantic model to predict late deliveries.

## Use Case
Predict which **open deliveries** will ship late relative to the Customer Requested Delivery Date, enabling:
- Proactive communication with business teams about specific late deliveries
- Identification of deliveries with high risk of shipping late
- Prioritization of corrective actions for strategic accounts

## This Notebook Will:
1. Connect to the semantic model
2. Load historical **closed deliveries** (last 2-4 weeks) for training
3. Validate the target variable: `AGE_REQ_DATE` (days late/early vs requested date)
4. Explore features: Plant, Brand, Channel, Carrier, Strategic Account, etc.
5. Prepare data for AutoML training

### 📦 1. Install Semantic Link

In [ ]:
%pip install -U semantic-link --q

### 🔧 2. Configuration

**IMPORTANT:** Update the semantic model name to match your environment.

In [ ]:
# ==============================================================================
# CONFIGURATION: Semantic Model Connection
# ==============================================================================
# This cell connects to your Power BI semantic model using Semantic Link.
# 
# WHY: Semantic Link allows Python to query Power BI semantic models directly,
#      avoiding data duplication and ensuring consistency with your reports.
#
# WHAT YOU NEED TO UPDATE:
# - DATASET: Change to match your exact semantic model name in Fabric
# ==============================================================================

import sempy.fabric as fabric 
import pandas as pd
from sempy.fabric import FabricDataFrame

# Semantic model name (UPDATE THIS to match your environment)
DATASET = "DLV Aging Columns & Measures"

# Get workspace ID (automatically detects current Fabric workspace)
ws = fabric.get_workspace_id()

print(f"✅ Workspace ID: {ws}")
print(f"✅ Semantic Model: {DATASET}")


### 📊 3. List Tables in Semantic Model

Let's see what tables are available in the semantic model.

In [ ]:
# List all tables in the semantic model
# WHY: Verify the semantic model is accessible and see available tables
tables_fdf = fabric.list_tables(DATASET, workspace=ws) 
print(f"Tables found: {len(tables_fdf)}")
tables_fdf


### 📋 4. List Columns in the Aging Table

Explore all columns available in the Aging table.

In [ ]:
# List all columns across all tables
# WHY: See what data is available in the semantic model
# NOTE: This shows table columns only - DAX measures are NOT included
columns_df = fabric.list_columns(DATASET, workspace=ws)
print(f"\nTotal columns across all tables: {len(columns_df)}")
print(f"\nColumn details:")
columns_df


### 🔗 5. Visualize Relationships

Check if there are any relationships between tables in the semantic model.

In [ ]:
# Check for relationships between tables
# WHY: Understanding table relationships helps with data modeling
# NOTE: Many semantic models have a single denormalized table (no relationships)
try:
    from sempy.relationships import plot_relationship_metadata
    relationships = fabric.list_relationships(workspace=ws, dataset=DATASET)
    
    if len(relationships) > 0:
        print(f"Found {len(relationships)} relationships")
        plot_relationship_metadata(relationships)
    else:
        print("No relationships found in this semantic model.")
        print("The Aging table appears to be a single fact table with all data.")
except Exception as e:
    print(f"Note: {e}")
    print("This is normal if the Aging table is a flat/denormalized table.")


### 📥 6. Load Closed Deliveries for Training

In [ ]:
# ==============================================================================
# LOAD TRAINING DATA: Closed Deliveries Only
# ==============================================================================
# WHY: Machine learning requires labeled data (known outcomes).
#      Closed deliveries have GI Date (ship date), so we know if they were late.
#
# FILTER: NOT(ISBLANK(Aging[GI Date])) 
#         This ensures we only get deliveries that have already shipped.
#
# NOTE: Open deliveries (no GI Date yet) will be scored in Notebook 03.
# ==============================================================================

dax_query = """
EVALUATE
FILTER(
    Aging,
    NOT(ISBLANK(Aging[GI Date]))
)
"""

df_closed = fabric.evaluate_dax(dataset=DATASET, dax_string=dax_query, workspace=ws)

# Clean column names (DAX adds table prefixes like 'Aging[column]')
df_closed.columns = [col.split('[')[-1].replace(']', '') if '[' in col else col for col in df_closed.columns]

print(f"✅ Loaded {len(df_closed):,} closed deliveries")
print(f"✅ Columns: {df_closed.shape[1]}")
df_closed.head()


### 🔍 7. Quick Data Check

### 🎯 9. Validate Target Variable: AGE_REQ_DATE

**Target Variable:** `AGE_REQ_DATE` - Days late/early relative to Customer Requested Delivery Date

- **Positive values** = Late delivery (shipped after requested date)
- **Zero** = On-time delivery  
- **Negative values** = Early delivery (shipped before requested date)

This is the key metric for:
- Meeting customer expectations
- Identifying deliveries at risk of SLA breach
- Bucketing late deliveries (0-2, 3-5, 6-9, 10+ days)

In [ ]:
# ==============================================================================
# VALIDATE TARGET VARIABLE: AGE_REQ_DATE
# ==============================================================================
# WHY: This is what we're trying to predict - how many days late/early.
#
# AGE_REQ_DATE = Days between actual ship date (GI Date) and 
#                customer requested delivery date
#
# Interpretation:
# - Positive values = Late delivery (shipped after requested date)
# - Zero = On-time delivery
# - Negative values = Early delivery (shipped before requested date)
# ==============================================================================

if 'AGE_REQ_DATE' in df_closed.columns:
    age_data = df_closed['AGE_REQ_DATE'].dropna()
    late_count = (age_data > 0).sum()
    on_time_count = (age_data <= 0).sum()
    
    print("="*60)
    print("TARGET VARIABLE: AGE_REQ_DATE")
    print("="*60)
    print(f"Total records: {len(age_data):,}")
    print(f"Mean: {age_data.mean():.2f} days")
    print(f"Median: {age_data.median():.2f} days")
    print(f"\nLate (>0): {late_count:,} ({late_count/len(age_data)*100:.1f}%)")
    print(f"On-time/Early (<=0): {on_time_count:,} ({on_time_count/len(age_data)*100:.1f}%)")
    print("="*60)
else:
    print("⚠️ AGE_REQ_DATE not found!")


### ✅ 8. Summary

Data is ready for model training!

In [ ]:
print("="*60)
print("DATA PREPARATION COMPLETE")
print("="*60)

# Validate target
if 'AGE_REQ_DATE' not in df_closed.columns:
    print("\n❌ ERROR: AGE_REQ_DATE not found!")
else:
    print("\n✅ TARGET VARIABLE: AGE_REQ_DATE")

# List available features
recommended_features = [
    'Plant', 'Brand', 'Channel', 'Product Category', 'Product Type',
    'Standard Or Custom', 'Credit Status', 'Distribution Status',
    'STRATEGIC_ACCOUNT', 'EWM Carrier Code', 'Shipping Point',
    'Delivery Type', 'DELIVERY_QTY', 'DELIVERY_VALUE_USD'
]

available = [f for f in recommended_features if f in df_closed.columns]
print(f"\n✅ Available features: {len(available)}/{len(recommended_features)}")
print(f"✅ Total records: {len(df_closed):,}")

print("\n" + "="*60)
print("NEXT: Open 02_autoML_training_pipeline.ipynb")
print("="*60)

---

## Next Step

Proceed to **`02_autoML_training_pipeline.ipynb`** to train the late delivery prediction model.

The model will predict:
- **AGE_REQ_DATE** (regression): How many days late/early will the delivery be?
- **is_late** (classification): Will the delivery be late (yes/no)?
- **lateness_bucket** (multi-class): Which bucket (0-2, 3-5, 6-9, 10+ days)?